# Import libraries

In [ ]:
import os
import csv
import time
import pickle
import numpy as np
import tensorflow as tf

from models.deep_crnn_model import *
from models.model_builders.create_dataset import *

# Model parameters

** Important editable parameters **

In [ ]:
# for now dataset must be one of
# iamHandwriting, BenthamDataset, or combined
dataset = "combined"
n_epochs = 5
batch_size = 16

** Less important parameters **

In [ ]:
data_folder = "../data/" + dataset
csv_file = data_folder + "/train.csv"
output_model_dir = "./tf_output/estimator/"
output_preds_dir = "./tf_output/prediction/"
output_graph_dir = "./tf_output/graph/"

optimizer='adam'
learning_rate=1e-3
learning_decay_rate=0.95
learning_decay_steps=5000

# load input_shape from file output by preprocess
with open(data_folder + "/img_size.txt", "r") as f:
    doc = f.readline()
    w, h = doc.split(",")
    input_shape = (int(float(h)), int(float(w)))
    print(input_shape)

# load alphabet from file output by preprocess
with open(data_folder + "/alphabet.txt", "r") as f:
    alphabet = f.readline()
    print(alphabet)

# Run model

** Create model ops **

In [ ]:
input_tensor = tf.placeholder(tf.float32, [None, input_shape[0], input_shape[1], 1])
labels = tf.placeholder(tf.string, [None])

# out = deep_crnn(input_tensor, labels, input_shape, alphabet, batch_size)
# train_op, loss_ctc, CER, accuracy, predictions_dict = out

In [ ]:
import tensorflow as tf
from models.model_builders.cnn_layers import *
from models.model_builders.lstm_layers import *
from models.model_builders.CTC_functions import *
from models.model_builders.create_optimizers import *
# from models.model_builders.md_lstm import *
import models.model_builders.md_lstm_ocr as network_utils



# def deep_cnn_mdlstm(input_tensor, labels, input_shape, alphabet, batch_size, 
#     optimizer="adam", is_training=True):
is_training=True
alphabet_codes = list(range(len(alphabet)))
n_classes = len(alphabet)
starting_filter_size = 16

In [ ]:
# seq_lens = network_utils.reshape(features["seq_lens"], [-1])
net = network_utils.conv2d(input_tensor, starting_filter_size, 3)
net = network_utils.max_pool2d(net, 2)
# seq_lens = network_utils.div(seq_lens, 2)
net = network_utils.mdlstm(net, starting_filter_size * 2, cell_type="LSTM")
net = network_utils.dropout(net, 0.25, is_training)
net = network_utils.conv2d(net, starting_filter_size * 3, 3)
net = network_utils.max_pool2d(net, 2)
net = network_utils.dropout(net, 0.25, is_training)
# seq_lens = network_utils.div(seq_lens, 2)
net = network_utils.mdlstm(net, starting_filter_size * 4, cell_type="LSTM")
net = network_utils.dropout(net, 0.25, is_training)
net = network_utils.conv2d(net, starting_filter_size * 5, 3)
net = network_utils.max_pool2d(net, 2)
net = network_utils.dropout(net, 0.25, is_training)
# seq_lens = network_utils.div(seq_lens, 2)
net = network_utils.mdlstm(net, starting_filter_size * 6, cell_type="LSTM")
net = network_utils.dropout(net, 0.25, is_training)
net = network_utils.conv2d(net, starting_filter_size * 7, 3)
net = network_utils.max_pool2d(net, 1)
net = network_utils.dropout(net, 0.25, is_training)
# seq_lens = network_utils.div(seq_lens, 2, is_floor=False)
net = network_utils.mdlstm(net, starting_filter_size * 8, cell_type="LSTM")
net = network_utils.dropout(net, 0.25, is_training)
net = network_utils.conv2d(net, starting_filter_size * 9, 3)
net = network_utils.max_pool2d(net, 1)
net = network_utils.dropout(net, 0.25, is_training)
# seq_lens = network_utils.div(seq_lens, 2, is_floor=False)
net = network_utils.mdlstm(net, starting_filter_size * 10, cell_type="LSTM")
net = network_utils.dropout(net, 0.25, is_training)
net = network_utils.collapse_to_rnn_dims(net)
lstm_out = network_utils.get_logits(inputs=net,
                                  num_classes=n_classes,
                                  num_steps=net.shape[1],
                                  num_hidden_units=net.shape[-1])

In [ ]:
# shape_tens = tf.shape(lstm_net)
# lstm_out = tf.reshape(fc_out, [shape_tens[0], -1, n_classes], name='reshape_out')  # [batch, width, n_classes]
# raw_pred = tf.argmax(tf.nn.softmax(lstm_out), axis=2, name='raw_prediction')
# Swap batch and time axis
# lstm_out = tf.transpose(lstm_out, [1, 0, 2], name='transpose_time_major')  # [width(time), batch, n_classes]

In [ ]:
predictions_dict = {'prob': lstm_out, 'raw_predictions': None}

In [ ]:
# loss layers
out = ctc_loss(predictions_dict, labels, input_shape, alphabet, 
    alphabet_codes, batch_size, n_pools=2*2*2*2*2)
loss_ctc, predictions_dict, CER, accuracy = out

# optimizer
train_op = create_optimizer(loss_ctc)

# return train_op, loss_ctc, CER, accuracy, predictions_dict

** Load dataset **

In [ ]:
out = create_iterator(csv_file, input_shape, batch_size)
dataset, iterator, next_batch, datasize = out

** Train model **

In [ ]:
saver = tf.train.Saver()

losses = []
cers = []
accs = []

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
#     saver.restore(sess, output_model_dir+"/model.ckpt")
    
    writer = tf.summary.FileWriter(output_graph_dir, sess.graph)
    n_batches = int(datasize / batch_size)
    for i in range(n_epochs):
        print("---------------------------------------------------------")
        print("Starting epoch", i)
        sess.run(iterator.initializer)
        for j in range(n_batches):
            input_tensor_b, labels_b = sess.run(next_batch)
            
            try:
                _, cer, acc, loss = sess.run([train_op, CER, accuracy, loss_ctc],
                             feed_dict={input_tensor: input_tensor_b, labels: labels_b})
                losses.append(loss)
                cers.append(cer)
                accs.append(accs)
                pickle.dump([cers, accs, losses], open(output_model_dir+"/metrics.pkl", "wb"))
                saver.save(sess, output_model_dir+"/model.ckpt")
                
                print('batch: {0}, loss: {3} \n\tCER: {1}, accuracy: {2}'.format(j, cer, acc, loss))
            except:
                print("Error at ", j)
        print('Avg Epoch time: {0} seconds'.format((time.time() - start_time)/(1.0*(i+1))))
        
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!') 